In [14]:
from kafka import KafkaProducer
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_io as tfio

from getConfig import getConfig

In [2]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

In [3]:
def write_to_kafka(topic_name, items):
    count=0
    producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
    for message, key in items:
        producer.send(topic_name,
                      key=key.encode('utf-8'),
                      value=message.encode('utf-8'))\
        .add_errback(error_callback)
        count+=1
    producer.flush()
    print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [4]:
def decode_kafka_item(item):
    message = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
    key = tf.strings.to_number(item.key)
    return (message, key)

In [5]:
NUM_COLUMNS=20
cfg = getConfig()
inputFile = cfg['dataLoc']+'clean.csv.gz'

In [6]:
iterator = pd.read_csv(inputFile,
                       chunksize=1e3)
rows = next(iterator)

In [8]:
train_df, test_df = train_test_split(rows, test_size=0.3, shuffle=True)
print("{} training rows and {} test".format(len(train_df), len(test_df)))

700 training rows and 300 test


In [9]:
trainX = train_df.drop(["label"], axis=1)
trainY = train_df["label"]

testX = test_df.drop(["label"], axis=1)
testY = test_df["label"]

In [10]:
# "filter(None)" removes any Null or empty rows
x_train = list(filter(None, trainX.to_csv(index=False).split("\n")[1:]))
y_train = list(filter(None, trainY.to_csv(index=False).split("\n")[1:]))

In [11]:
write_to_kafka("iot-train", zip(x_train, y_train))

Wrote 700 messages into topic: iot-train


In [12]:
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
train_ds = tfio.IODataset.from_kafka('iot-train', partition=0, offset=0)

train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)

2023-01-10 15:19:58.943848: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-01-10 15:19:58.944142: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2023-01-10 15:19:59.065794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-10 15:19:59.065848: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-10 15:19:59.065868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-1): /proc/driver/nvidia/version does not exist
2023-01-10 15:19:59.066241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary

In [15]:
train_ds = train_ds.map(decode_kafka_item)

In [28]:
train_ds = train_ds.batch(BATCH_SIZE)

In [30]:
OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

In [31]:
# design/build the model
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(NUM_COLUMNS,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2688      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [32]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [33]:
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10


/home/tbrownex/miniconda3/envs/tf/lib/python3.9/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-01-10 15:30:33.656015: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 0


InvalidArgumentError: Graph execution error:

Field 1 in record 0 is not a valid float: CHP1KH2KbxafIuiSj2
	 [[{{node DecodeCSV}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_1507]